## Objective : We will measure performance of final model .

In [2]:
#!pip install  scikit-plot

In [3]:
#import libraries here; add more as necessary.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import sidetable
import warnings
import joblib
warnings.filterwarnings("ignore")

#Models
from xgboost import XGBClassifier


from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from pandas import ExcelWriter
from openpyxl import load_workbook

%load_ext autoreload
%autoreload 2

## Load Data

In [4]:
X_train = pd.read_csv(r'C:\Users\psahu\Documents\My_projects\online_shoppers\data\Transformed_X_train_shoppersData.csv')
y_train = pd.read_csv(r'C:\Users\psahu\Documents\My_projects\online_shoppers\data\Transformed_y_train.csv')

In [5]:
X_test = pd.read_csv(r'C:\Users\psahu\Documents\My_projects\online_shoppers\data\Transformed_X_test_shoppersData.csv')
y_test = pd.read_csv(r'C:\Users\psahu\Documents\My_projects\online_shoppers\data\Transformed_y_test.csv')

In [6]:
X_val = pd.read_csv(r'C:\Users\psahu\Documents\My_projects\online_shoppers\data\Transformed_X_val_shoppersData.csv')
y_val = pd.read_csv(r'C:\Users\psahu\Documents\My_projects\online_shoppers\data\Transformed_y_val.csv')

In [7]:
import Functions_File as my_ff

Functions_File is successfully imported.


# Final Model

In [8]:
print(X_train.shape ,y_train.shape)

(7398, 68) (7398, 1)


In [9]:
print(X_test.shape ,y_test.shape)

(2466, 68) (2466, 1)


In [10]:
print(X_val.shape ,y_val.shape)

(2466, 68) (2466, 1)


## Before Sampling

In [30]:
y_train.value_counts()

Revenue
0          6271
1          1127
dtype: int64

In [31]:
#USing SMOTE for oversampling of Minority class
smt = SMOTE(sampling_strategy = 1)
X_train, y_train = smt.fit_sample(X_train, y_train)

## After Sampling

In [32]:
y_train.value_counts()

Revenue
1          6271
0          6271
dtype: int64

## XGBoost

In [49]:
model =  XGBClassifier(colsample_bytree=0.7, gamma=0.0,
                               learning_rate=0.05, max_depth=4,
                               min_child_weight=3, random_state=50)

In [51]:
model.fit(X_train,y_train)

XGBClassifier(colsample_bytree=0.7, gamma=0.0, learning_rate=0.05, max_depth=4,
              min_child_weight=3, random_state=50)

## Prediction on test set.  

In [52]:
y_pred_test = model.predict(X_test)
print("Accuracy:" , accuracy_score(y_test,y_pred_test)*100)

Accuracy: 89.13219789132198


In [53]:
y_pred_test_prob=model.predict_proba(X_test)

In [54]:
confusion_matrix(y_test, y_pred_test)

array([[1910,  174],
       [  94,  288]], dtype=int64)

In [55]:
from sklearn.metrics import recall_score
recall_score(y_test, y_pred_test,  average='binary')

0.7539267015706806

In [56]:
#Compute precision, recall, F-measure and support

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.95      0.92      0.93      2084
           1       0.62      0.75      0.68       382

    accuracy                           0.89      2466
   macro avg       0.79      0.84      0.81      2466
weighted avg       0.90      0.89      0.90      2466



##  Prediction on validation set 

In [57]:
y_pred_val = model.predict(X_val)
print("Accuracy:" , accuracy_score(y_val,y_pred_val)*100)

Accuracy: 88.84833738848337


In [58]:
recall_score_val=recall_score(y_val,y_pred_val,  average='binary')
recall_score_val

0.7393483709273183

In [59]:
y_pred_val_proba =model.predict_proba(X_val)

In [60]:
#
X_val_probs = model.predict_proba(X_val)[:,1]
#%% Confusion metrics - validation set
confusion_matrix(y_val, y_pred_val)

array([[1896,  171],
       [ 104,  295]], dtype=int64)

In [61]:
#%% Compute precision, recall, F-measure and support

from sklearn.metrics import classification_report
print(classification_report(y_val, y_pred_val))

              precision    recall  f1-score   support

           0       0.95      0.92      0.93      2067
           1       0.63      0.74      0.68       399

    accuracy                           0.89      2466
   macro avg       0.79      0.83      0.81      2466
weighted avg       0.90      0.89      0.89      2466



## Feature Importance

In [62]:
xgb_feat_imp_cover = model.get_booster().get_score(importance_type='cover')
xgb_feat_imp_gain = model.get_booster().get_score(importance_type='gain')
xgb_feat_imp_weight = model.get_booster().get_score(importance_type='weight')

In [63]:
full_feat_imp = pd.DataFrame( [ xgb_feat_imp_gain , xgb_feat_imp_cover, xgb_feat_imp_weight]).T
full_feat_imp.columns = [ 'gain' ,'cover' ,  'weight']

In [64]:
#Sort as per 'gain'
full_feat_imp.sort_values("gain", axis = 0, ascending = False , inplace = True)

In [65]:
full_feat_imp=full_feat_imp.reset_index().rename(columns = {'index' : 'features'})
full_feat_imp

,features,gain,cover,weight
0,PageValues,627.784797,957.683868,127.0
1,OperatingSystems_3,85.584044,926.996185,29.0
2,Month_May,77.801530,1034.404361,79.0
3,Month_Dec,76.295721,817.625033,55.0
4,TrafficType_3,65.803403,1064.477347,21.0
5,Month_Mar,60.987730,810.174129,65.0
6,OperatingSystems_4,58.052595,529.736290,4.0
7,VisitorType_Returning_Visitor,57.353001,801.156855,70.0
8,OperatingSystems_2,54.827707,598.363151,31.0
9,Region_2,49.646236,842.753483,14.0
